# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

March-June 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).
* add definition of fields to list in Introduction

# Note
Currently the code will fetch a large zip file from ORACC, download it, extract certain files from the zip file and parse those. The zip file contains all data that belong to an ORACC project or sub-project. Since one may run this notebook several times for collecting data from the same project, this may not be the best process (the download will take place every time). Move the download process to a separate notebook, preceding the current one.


# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) (Open Richly Annotated Cuneiform Corpus) JSON files that contain textual data and produce a `.csv` file in the directory `data/raw` with the relevant data for use in the phylogenetics project. The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file). For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file includes various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `requests` library.  

This notebook is written for **Python 3.5** with **Pandas 0.19** and **requests 2.18.1**.

The notebook was written for the [Digital Humanities Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). The particular data selection and data manipulation performed in this notebook are inspired by the needs of that project (for instance, non-Sumerian words are filtered out). It should be fairly easy to adapt the notebook to the purposes of any other project that wishes to use [ORACC](http://oracc.org) data.

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [ ]:
import pandas as pd   
import requests
import zipfile
import tqdm

# Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `data/text_ids`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs (not in a word processor such as MS Word), and the filename should end in `.txt`.

In [ ]:
filename = input('Filename: ')

In [ ]:
textids = '../data/text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]
pqxnos[:5]

# Parse
The function `parsejson()` takes one argument (the `textid`).  

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The resulting dictionary includes keys such as `lang` (for language), `guideword`, `sense`, etc. - all the elements that define an [ORACC](http://oracc.org) signature. The dictionary also includes the key `id_word` (a sequential number for each word in each line) that allows the user to reassemble a text in the original word and line order.

In [ ]:
def parsejson(text, lemm_l = None):
    label = ''
    if lemm_l == None:
        lemm_l = []
    for dictionary in text["cdl"]:
        if "cdl" in dictionary: 
            parsejson(dictionary, lemm_l)
        if "label" in dictionary:
            label = dictionary["label"]
        if "f" in dictionary:
            lemma = dictionary["f"]
            lemma["id_word"] = dictionary["ref"]
            lemma["label"] = label
            lemm_l.append(lemma)
        if "strict" in dictionary and dictionary["strict"] == "1":
            keys = ["extent", "scope","state"]
            lemma = {key: dictionary[key] for key in keys}
            lemma["id_word"] = dictionary["ref"] + ".0"
            lemm_l.append(lemma)
    return lemm_l

# Call the Parser Function for Each Textid

In [ ]:
word_l = []
for id_text in pqxnos:
    project = id_text[:-8].lower()
    pqx = id_text[-7:].upper()
    url = "http://oracc.org/" + project + '/corpusjson/' + pqx + '.json'
    r = requests.get(url)
    try:
        text = r.json()
        print("parsing " + id_text)
        lemm_text = parsejson(text)
        for lemm in lemm_text:
            lemm['textid'] = id_text
        word_l.extend(lemm_text)
    except:
        print(id_text + ' is not available or not complete')

# Transform the Data into a DataFrame

If a text has no breakage information in the form of `$ 1 line broken` (etc.) the fields `extent`, `scope`, and `state` do not exist. The fields `extent` and `scope` are referenced in the code below. After creating the dataframe the existence of these two fields is checked - if they do not exist, empty columns are created.

In [ ]:
words = pd.DataFrame(word_l)
if not 'extent' in words.columns:
    words['extent'] = ''
if not 'scope' in words.columns:
    words['scope'] = ''
words

# Remove Spaces and Commas from Guide Word and Sense
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [ ]:
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization, `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Sumerian words

In [ ]:
lang = ['sux', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [ ]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]

## Group by Line
Create a `line_id` from `word_id`. The field `word_id` has the format `Q000039.76.1` (first word in line 76 of Q000039). The corresponding `line_id` is `Q000039.76`. 

In [ ]:
words["id_line"] = [item[:item.rfind(".")] for item in words["id_word"]]

In [ ]:
df = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
df        

# Create fields `id_text` and `line`
The field `id_text` is the P-number of the document and is equal to the first seven characters of `id_line`. The field `line` is an artificial line counter (integer) that is used to keep things in the right order. It derives from `id_line` by keeping eveything after the last dot (note that the regular expression `.+\.` is greedy and will match the longest possible string ending in a dot).

In [ ]:
pd.options.mode.chained_assignment = None # this prevents Pandas from issuing warnings about chained assignments
df['id_text'] = df['id_line'].str[:7]
df['line'] = [int(line[line.rfind(".")+1:]) for line in df['id_line']] #create a line number for sorting
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df

## Save in CSV Format

In [ ]:
filename = '../data/raw/' + filename[:-4] + '.csv'
print('saving ' + filename)
with open(filename, 'w') as w:
    df.to_csv(w, encoding='utf8')